In [ ]:
# Notebook for Testing

In [2]:
from music21 import converter, instrument, note, chord, stream
from mido import MidiFile
import numpy
import glob
import pickle
import numpy as np

In [12]:
import pygame

pygame.init()
pygame.mixer.music.load(r'test_output.mid')
pygame.mixer.music.play()

# while pygame.mixer.music.get_busy():
#     pygame.time.wait(1000)
pygame.time.wait(10000)
pygame.mixer.music.stop()

In [3]:


def get_notes(filename):
    buffer_size = 500000
    notes = [''] * buffer_size
    Last_time = 0
    offset = []
    midi = converter.parse(filename)
    
    print('parsing {}'.format(filename))
    try:  # file has instrument parts
        s2 = instrument.partitionByInstrument(midi)
        notes_to_parse = s2.recurse()
    except:  # file has notes in a flat structure
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
      if isinstance(element, note.Note):
        
         time = int(element.offset * 4)
        # print(element)
         if notes[time] == '':
            notes[time] = str(element.pitch.midi)
            last_time = time
      elif isinstance(element, chord.Chord):
         time = int(element.offset * 4)
         if notes[time] == '':
            notes[time] = '.'.join(str(n) for n in element.normalOrder)
            last_time = time

    
    return notes[:last_time]

notes = get_notes("midi\kaggle\mozart\mz_311_1.mid")


parsing midi\kaggle\mozart\mz_311_1.mid


In [4]:
def prepare_sequences(notes, n_vocab, note_to_int):
   sequence_length = 400
   network_input = []
   network_output = []
   for i in range(0, len(notes) - sequence_length, 1):
      sequence_in = notes[i:i + sequence_length]
      sequence_out = notes[i + sequence_length]
      network_input.append([note_to_int[char] for char in sequence_in])
      network_output.append(note_to_int[sequence_out])
   n_patterns = len(network_input)
   normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
   normalized_input = normalized_input / float(n_vocab)
   # network_output = to_categorical(network_output)
   return (network_input, normalized_input, network_output)

In [5]:
pitchnames = sorted(set(item for item in notes))  
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

network_input, normalized_input, network_output = prepare_sequences(notes, len(set(notes)), note_to_int)

In [6]:
def make_markov_chain(seq):
    n = len(seq)
    transition_matrix = np.zeros((108,108))
    for i in range(1,n):
        
        transition_matrix[seq[i-1],seq[i]] += 1
    for i in range(108):
        transition_matrix[i] /= np.sum(transition_matrix[i])
        rem = 1 - np.sum(transition_matrix[i])
        transition_matrix[i][0] += rem
    
    return (transition_matrix)
    


In [7]:
def random_walk(transition_matrix):
    generated_seq = []
    states = np.arange(0,108,1)
    f = 20
    generated_seq.append(f)
    for i in range(400):
        state = np.random.choice(states, p=transition_matrix[f])
        f = state
        generated_seq.append(f)
    return (generated_seq)


transition_matrix = make_markov_chain(network_input)
generated_sequence = random_walk(transition_matrix)


C:\Users\mohit\AppData\Local\Temp\ipykernel_43268\1497260535.py:8: RuntimeWarning: invalid value encountered in divide
  transition_matrix[i] /= np.sum(transition_matrix[i])


In [8]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """

    output_notes = []

    # create note and chord objects based on the values generated by the model
    for idx, pattern in enumerate(prediction_output):
      if len(pattern) != 0:
        notes_in_chord = [int(n) for n in pattern.split('.')]
        new_chord = chord.Chord(notes_in_chord)
        new_chord.offset = idx/4
        output_notes.append(new_chord)
        # increase offset each iteration so that notes do not stack
    print(output_notes)
    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')

sequence_notes = []

for i in generated_sequence:
   
   if i not in int_to_note:
      sequence_notes.append('')
   else:
      sequence_notes.append(int_to_note[i])

create_midi(sequence_notes)

[<music21.chord.Chord E F#>, <music21.chord.Chord E F#>, <music21.chord.Chord E- F->, <music21.chord.Chord A5>, <music21.chord.Chord C#5>, <music21.chord.Chord B4>, <music21.chord.Chord C#5>, <music21.chord.Chord F# A>, <music21.chord.Chord A4>, <music21.chord.Chord E4>, <music21.chord.Chord F#4>, <music21.chord.Chord E F#>, <music21.chord.Chord D E>, <music21.chord.Chord C# D>, <music21.chord.Chord E5>, <music21.chord.Chord C#5>, <music21.chord.Chord A4>, <music21.chord.Chord B3>, <music21.chord.Chord F#3>, <music21.chord.Chord G3>, <music21.chord.Chord A3>, <music21.chord.Chord B3>, <music21.chord.Chord F#3>, <music21.chord.Chord E5>, <music21.chord.Chord F#5>, <music21.chord.Chord E5>, <music21.chord.Chord A4>, <music21.chord.Chord G#4>, <music21.chord.Chord E F# G#>, <music21.chord.Chord F#5>, <music21.chord.Chord E5>, <music21.chord.Chord E4>, <music21.chord.Chord B5>, <music21.chord.Chord E4>, <music21.chord.Chord E2>, <music21.chord.Chord E4>, <music21.chord.Chord G#5>, <music21